In [1]:
import os
import pandas as pd
import pickle


#exp_name = 'preprocessing_type'
#experiments = ["adam", "other", "rmsprop", "sgd"]
logs_path = os.path.join('logs', 'general_multi_layer_100_pending')

grid_search_results = []
results_path = os.path.join(logs_path, 'gridsearch_results.pkl')
with open(results_path, 'rb') as f:
    partial_results = pickle.load(f)
grid_search_results.extend(partial_results)
print(grid_search_results[0].keys())
    
    

dict_keys(['model-rnn_cell_params-units', 'model-bidirectional', 'model-num_layers', 'model-dilation_base', 'model-residual_block_size', 'model-smyl_std', 'model-smyl_residual', 'metrics_per_fold'])


In [2]:
import pickle

with open(os.path.join(logs_path, 'basic_parameters.pickle'), 'rb') as f:
    basic_parameters = pickle.load(f)
print(basic_parameters)

{'dataset': {'data_path': '/Users/maderajan/Programming/LSTM-crystal-growth/python/data', 'scaler': StandardScaler(), 'preprocessing_function': <function lag_features at 0x3133f36a0>, 'shuffle': True, 'final_testing': False, 'test_size': 0.0, 'outputs_lag': 3, 'inputs_lag': 0, 'window_size': 3, 'return_sequences': True, 'seed': 42, 'scaler_partitions': [[0, 1], [2, 3, 4, 5, 6], [7]], 'scaler_kwargs': {}, 'k_folds': 10}, 'dataset_class': <class 'data_processing.dataset.CrystalDataset'>, 'model': {'bidirectional': False, 'cell_type': 'lstm', 'rnn_cell_params': {'units': 20, 'peephole_connections': False, 'layer_normalization': False, 'ln_epsilon': 0.001, 'ln_center': True, 'ln_scale': True, 'ln_beta_initializer': 'zeros', 'ln_gamma_initializer': 'ones', 'dropout': 0.0, 'recurrent_dropout': 0.0}, 'num_layers': 1, 'rnn_layer_params': {'return_sequences': True}, 'dilation_base': None, 'residual_block_size': None, 'smyl_std': False, 'smyl_residual': False}, 'training': {'batch_size': 32, 'ep

In [3]:
print(len(grid_search_results))
for exp in grid_search_results:
    for key in exp.keys():
        if key != 'metrics_per_fold':
            print(f"{key}: {exp[key]}")
    #print(f"epochs: {exp["training-epochs"]}")
    folds = exp["metrics_per_fold"]
    for i, fold in enumerate(folds):
        print(f"\tFold {i+1}:")
        for metric, values in fold.items():
            scaled_val = values["scaled"]
            unscaled_val = values["unscaled"]
            print(f"\t\t{metric}: {scaled_val.metric:.6f} ({unscaled_val.metric:.6f})\n\t\t\tper_sample s (u):\n\t\t\t\t {scaled_val.metric_per_sample}\n\t\t\t\t({unscaled_val.metric_per_sample})")
    print()
    print("######################################")

9
model-rnn_cell_params-units: 50
model-bidirectional: False
model-num_layers: 2
model-dilation_base: None
model-residual_block_size: 2
model-smyl_std: False
model-smyl_residual: True
	Fold 1:
		mean_squared_error: 0.000011 (0.483654)
			per_sample s (u):
				 [7.5505163e-06, 5.092761e-06, 7.5467233e-06, 5.1116936e-06, 1.9091904e-05, 1.0555104e-05, 5.0916724e-06, 1.2599749e-05, 5.105313e-06, 5.9509134e-06, 1.7351193e-05, 1.910934e-05, 4.4643393e-06, 5.1308284e-06, 1.30056505e-05, 1.3312159e-05, 6.5685126e-06, 1.4312071e-05, 5.2010755e-06, 6.58359e-06, 4.616552e-06, 5.8431765e-06, 5.1522525e-06, 5.9984077e-06, 5.189874e-06, 3.2342297e-05, 2.012348e-05, 1.4567833e-05, 2.1532205e-05, 1.913393e-05, 1.7768034e-05, 5.10569e-06, 1.1017331e-05, 5.3704393e-06, 5.445552e-06, 2.7261674e-05, 5.396408e-06, 5.3905046e-06, 2.5098545e-05, 5.5474584e-06, 3.3768905e-05, 5.2534047e-06, 1.8503191e-05, 1.5104451e-05, 5.139362e-06]
				([0.34540984, 0.2722139, 0.3348795, 0.2719672, 0.8250006, 0.4665425, 0.2

1. 
   - For each experiment, get all samples. 
   - The samples are ordered for each experiment in the same way. 

In [4]:
from sklearn.preprocessing import FunctionTransformer
from types import FunctionType
import re

def get_exp_name(exp):
    name = ""
    for i, key in enumerate(exp.keys()):
        if key != 'metrics_per_fold':
            if i > 0:
                name += " "
            if isinstance(exp[key], str):
                name += exp[key]
            elif isinstance(exp[key], FunctionTransformer):
                return exp[key].func.__name__
            elif isinstance(exp[key], bool):
                name += key if exp[key] else "not "+ key
            elif isinstance(exp[key], FunctionType):
                name += exp[key].__qualname__
            else:
                key_parts = key.split("-")
                last_part = key_parts[-1]
                key_words = last_part.split("_")
                shortcut = "".join([word[0] for word in key_words]) if len(key_words) > 1 else key_words[0]
                name += shortcut + "=" + str(exp[key])

    #name = name.replace("dataset-", "").replace("_", " ").replace("-", " ")
    name = name.capitalize()
    return name

exp_samples = {}
for i, exp in enumerate(grid_search_results):
    exp_name = get_exp_name(exp)
    exp_samples[exp_name] = []
    folds = exp["metrics_per_fold"]
    for fold in folds:
        exp_samples[exp_name].extend(fold["root_mean_squared_error"]["unscaled"].metric_per_sample)
    #print(len(exp_samples[exp_name]))

#for exp_name, samples in exp_samples.items():
#    print(f"{exp_name}: {len(samples)}")
#    print(f"mean: {sum(samples)/len(samples)}")
#    print(f"std: {pd.Series(samples).std()}")
    


In [5]:
from scipy import stats
import statsmodels.stats.multitest
from critdd import Diagram
from itertools import combinations
import numpy as np
import pandas as pd

samples = list(exp_samples.values())
print(np.array(samples).shape)
friedman_result = stats.friedmanchisquare(*samples)
friedman_result

(9, 450)


FriedmanchisquareResult(statistic=669.0091851851848, pvalue=3.353181189241689e-139)

In [6]:
combs = combinations(samples, 2)

pvals = []
for comb in combs:
    pvals.append(stats.wilcoxon(comb[0], comb[1]).pvalue)

pvals = np.array(pvals)
pvals.sort()
print(pvals)

for i in range(len(pvals)):
    if pvals[i] > 0.05/(len(pvals)-i):
        print(f"{pvals[i]} is not st. significant")
        print(f"{0.05/(len(pvals)-i)} is the threshold")
        break
    else:
        print(f"{pvals[i]} is st. significant")

statsmodels.stats.multitest.multipletests(pvals, method="holm")

[1.70691340e-57 5.75745330e-55 1.23070896e-52 1.06887160e-35
 7.68682589e-31 3.95369882e-30 2.51949499e-29 6.65641760e-28
 1.35909848e-27 1.40301995e-27 4.01920488e-26 1.49188032e-23
 8.85556591e-21 3.17821627e-19 3.25216736e-19 3.68428403e-19
 5.14098371e-17 2.96121629e-15 4.05465966e-14 4.58266237e-14
 1.94649084e-10 3.33082572e-10 7.09126503e-09 8.43200877e-08
 6.51679623e-05 2.71060657e-04 4.19128777e-04 1.67645101e-03
 8.60244627e-03 1.80748233e-02 2.71729461e-02 1.07202804e-01
 1.28429764e-01 2.90694101e-01 4.88392756e-01 5.09318253e-01]
1.7069133995228536e-57 is st. significant
5.7574533014925946e-55 is st. significant
1.2307089592027859e-52 is st. significant
1.0688716003199598e-35 is st. significant
7.686825890694547e-31 is st. significant
3.95369882022661e-30 is st. significant
2.5194949872157935e-29 is st. significant
6.656417595898523e-28 is st. significant
1.359098484863048e-27 is st. significant
1.4030199460146753e-27 is st. significant
4.0192048810700657e-26 is st. signi

(array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True, False, False, False, False, False, False, False, False]),
 array([6.14488824e-56, 2.01510866e-53, 4.18441046e-51, 3.52727628e-34,
        2.45978429e-29, 1.22564663e-28, 7.55848496e-28, 1.93036110e-26,
        3.80547576e-26, 3.80547576e-26, 1.04499327e-24, 3.72970080e-22,
        2.12533582e-19, 7.30989742e-18, 7.30989742e-18, 7.73699646e-18,
        1.02819674e-15, 5.62631095e-14, 7.29838740e-13, 7.79052603e-13,
        3.11438534e-09, 4.99623859e-09, 9.92777104e-08, 1.09616114e-06,
        7.82015548e-04, 2.98166723e-03, 4.19128777e-03, 1.50880591e-02,
        6.88195702e-02, 1.26523763e-01, 1.63037676e-01, 5.36014018e-01,
        5.36014018e-01, 8.72082302e-01, 9.76785513e-01, 9.76785513e-01]),
 0.0014237991678133222,
 0.001388888888888889)

In [7]:
diag_df = pd.DataFrame(exp_samples)
diag_df.columns

Index(['Units=50 not model-bidirectional nl=2 db=none rbs=2 not model-smyl_std model-smyl_residual',
       'Units=50 not model-bidirectional nl=4 db=none rbs=2 not model-smyl_std model-smyl_residual',
       'Units=50 not model-bidirectional nl=6 db=none rbs=2 not model-smyl_std model-smyl_residual',
       'Units=100 not model-bidirectional nl=2 db=none rbs=2 not model-smyl_std model-smyl_residual',
       'Units=100 not model-bidirectional nl=4 db=none rbs=2 not model-smyl_std model-smyl_residual',
       'Units=100 not model-bidirectional nl=6 db=none rbs=2 not model-smyl_std model-smyl_residual',
       'Units=200 not model-bidirectional nl=2 db=none rbs=2 not model-smyl_std model-smyl_residual',
       'Units=200 not model-bidirectional nl=4 db=none rbs=2 not model-smyl_std model-smyl_residual',
       'Units=200 not model-bidirectional nl=6 db=none rbs=2 not model-smyl_std model-smyl_residual'],
      dtype='object')

In [8]:
exp_samples.values()

dict_values([[0.58771574, 0.5217414, 0.5786863, 0.5215061, 0.9082959, 0.68303865, 0.5232931, 0.682202, 0.3562369, 0.5302598, 0.7805194, 0.91878796, 0.3995419, 0.4642239, 0.6834745, 0.6850802, 0.52293605, 0.6941961, 0.51847297, 0.5225764, 0.41250324, 0.5344366, 0.52107704, 0.528081, 0.47021917, 1.2272109, 0.9354579, 0.7312335, 0.9570949, 0.9162307, 0.7748401, 0.5223788, 0.6837732, 0.53187174, 0.3693171, 1.1305723, 0.3640634, 0.53050286, 1.082872, 0.5339859, 1.1614779, 0.5259331, 0.9013157, 0.7221023, 0.5204253, 0.4115911, 0.43700397, 0.30612493, 0.75288755, 0.29095435, 0.6409497, 0.80209583, 0.29161704, 0.3674138, 0.8095733, 0.821121, 0.8039159, 0.38020965, 0.30577868, 0.73973215, 0.4661122, 0.46063834, 0.3026708, 0.8269805, 0.7203804, 0.7498268, 0.3027611, 0.7987689, 0.4955463, 0.80248713, 0.8004979, 0.670974, 0.7366022, 0.77949417, 0.7329874, 0.4246372, 0.43450055, 0.42676997, 0.6079608, 0.46345654, 0.5766286, 0.42012474, 0.4578669, 0.45182014, 0.6922278, 0.46213117, 0.33038354, 0.339

In [9]:
x=np.array(list(exp_samples.values()))
treatment_names = list(exp_samples.keys())
print(treatment_names)
x = x.T
diagram = Diagram(x,
    treatment_names=treatment_names,
    maximize_outcome = False,
)

diagram.to_file(
    "critdd_100_and_more.tex",
    alpha=0.05,
    adjustment="holm",
    reverse_x=True,
)

['Units=50 not model-bidirectional nl=2 db=none rbs=2 not model-smyl_std model-smyl_residual', 'Units=50 not model-bidirectional nl=4 db=none rbs=2 not model-smyl_std model-smyl_residual', 'Units=50 not model-bidirectional nl=6 db=none rbs=2 not model-smyl_std model-smyl_residual', 'Units=100 not model-bidirectional nl=2 db=none rbs=2 not model-smyl_std model-smyl_residual', 'Units=100 not model-bidirectional nl=4 db=none rbs=2 not model-smyl_std model-smyl_residual', 'Units=100 not model-bidirectional nl=6 db=none rbs=2 not model-smyl_std model-smyl_residual', 'Units=200 not model-bidirectional nl=2 db=none rbs=2 not model-smyl_std model-smyl_residual', 'Units=200 not model-bidirectional nl=4 db=none rbs=2 not model-smyl_std model-smyl_residual', 'Units=200 not model-bidirectional nl=6 db=none rbs=2 not model-smyl_std model-smyl_residual']


In [29]:
def get_models_without_dropout(diagram: Diagram):
    """Gets the best optimizer for each type (Adam, RMSprop, SGD, ...)"""
    sorted_treatments = sorted(zip(diagram.treatment_names, diagram.average_ranks), key=lambda x: x[1])
    no_dropout_models = {}
    for name, avg_rank in sorted_treatments:
        if name.endswith("dropout=0.0"):
            #name = re.sub(" dropout=0.0", "", name)
            no_dropout_models[name] = (name, avg_rank)
    return no_dropout_models

def get_partial_diagram(subset_names, samples):
    x = np.array([samples[name] for name in subset_names])
    x = x.T
    return Diagram(x,
        treatment_names=subset_names,
        maximize_outcome = False,
    )

no_dropout_models = get_models_without_dropout(diagram)
partial_diagram = get_partial_diagram([name for name, _ in no_dropout_models.values()], exp_samples)

print("Previous average ranks:")
for name, avg_r in no_dropout_models.values():
    print(f"{name}: {avg_r}")

print("\nNew average ranks:")
sorted_treatments = sorted(zip(partial_diagram.treatment_names, partial_diagram.average_ranks), key=lambda x: x[1])
for name, avg_r in sorted_treatments:
    print(f"{name}: {avg_r}")

partial_diagram.to_file(
    "critdd_layernorm_no_drop.tex",
    alpha=0.05,
    adjustment="holm",
    reverse_x=True,
)

Previous average ranks:
Not model-layer_normalization rd=0.0 dropout=0.0: 4.302222222222222
Model-layer_normalization rd=0.2 dropout=0.0: 4.377777777777778
Model-layer_normalization rd=0.1 dropout=0.0: 4.988888888888889
Model-layer_normalization rd=0.0 dropout=0.0: 5.015555555555555
Not model-layer_normalization rd=0.2 dropout=0.0: 5.3933333333333335
Model-layer_normalization rd=0.05 dropout=0.0: 5.595555555555555
Model-layer_normalization rd=0.3 dropout=0.0: 6.0311111111111115
Not model-layer_normalization rd=0.3 dropout=0.0: 6.153333333333333
Not model-layer_normalization rd=0.1 dropout=0.0: 6.388888888888889
Not model-layer_normalization rd=0.05 dropout=0.0: 6.753333333333333

New average ranks:
Not model-layer_normalization rd=0.0 dropout=0.0: 4.302222222222222
Model-layer_normalization rd=0.2 dropout=0.0: 4.377777777777778
Model-layer_normalization rd=0.1 dropout=0.0: 4.988888888888889
Model-layer_normalization rd=0.0 dropout=0.0: 5.015555555555555
Not model-layer_normalization rd